# PDF Text Extractor Utility

> Extracts unstructured text from scientific papers published as PDF files .

In [ ]:
#| default_exp utils.pdf_research_article_text_extractor

In [ ]:
#| export

import dataclasses
from langchain.document_loaders.pdf import BasePDFLoader
from langchain.document_loaders.base import BaseBlobParser
from langchain.docstore.document import Document
from langchain.document_loaders.blob_loaders import Blob
import re
from typing import Optional, List, Iterator, Mapping, Any, Dict
from dataclasses import field
import requests
import nltk


In [ ]:
#| export

@dataclasses.dataclass
class CumulativeTextFeature:
    name:str = ''
    d:dict = field(default_factory=dict)

    def __init__(self, name:str):
        self.name = name
        self.d = {}

    def add(self, feat:str, txt:str):
        if self.d.get(feat) is None:
            self.d[feat] = len(txt)
        else:
            self.d[feat] += len(txt)   

    def read_most_common(self):
        return max(self.d, key=self.d.get)

    def read_next_common(self):
        keys = sorted(self.d, key=self.d.get, reverse=True)
        if len(keys) > 1:
            return keys[1]
        else:
            return None
    
    def read_distribution(self):
        return self.d

    def read_total_length(self):
        return sum(self.d.values())

    def read_normalized_distribution(self):
        tl = self.read_total_length()
        return {k:self.d[k]/tl for k in self.d.keys()}

@dataclasses.dataclass
class LAPDF_FeatureBlock():
    """A block of text with spatial features occuring in a PDF full-text article."""

    page: int

    x0:float
    x1:float
    midx:float
    width:float

    y0:float
    y1:float
    midy:float
    height:float
    
    nlines:int
    sizes:CumulativeTextFeature = field(default_factory=CumulativeTextFeature)
    fonts:CumulativeTextFeature = field(default_factory=CumulativeTextFeature)
    
    text:str = ''
    t:str = ''

    def __init__(self, p:int, x0:float, y0:float, x1:float, y1:float, text:str, 
                 nlines:int, sizes:dict, fonts:dict, pos_err:float=0.05):
        self.x0 = x0
        self.y0 = y0
        self.x1 = x1
        self.y1 = y1
        self.midx = (x0 + x1)/2.0
        self.width = x1 - x0
        self.midy = (y0 + y1)/2.0
        self.height = y1 - y0
        self.page = p
        self.text = text
        self.nlines = nlines
        self.t = re.sub('\-\n','',self.text)
        self.t = re.sub('\n',' ',self.text)
        self.t = re.sub('\s+',' ',self.text)
        self.sizes = sizes
        self.fonts = fonts
        self.pos_err = pos_err

    def generate_layout_features(self, min_x0:float, min_y0:float, max_x1:float, max_y1:float):
        d_center = (min_x0 + max_x1)/2.0 - self.midx
        d_rcenter = (min_x0 + max_x1)/2.0 - self.x1
        d_lcenter = self.x0 - (min_x0 + max_x1)/2.0 
        d_lmargin = min_x0 - self.x0
        d_rmargin = self.x1 - max_x1
        d_top = self.x0 - min_y0 
        d_bottom = max_y1 - self.x0
        top_size = self.sizes.read_most_common()
        dist = self.sizes.read_normalized_distribution()
        size_prop = dist.get(top_size)
        top_font = self.fonts.read_most_common()
        dist2 = self.fonts.read_normalized_distribution()
        font_prop = dist2.get(top_font)
        return {
            'page': self.page,
            'd_center': d_center,
            'd_rcenter': d_rcenter,
            'd_lcenter': d_lcenter, 
            'd_lmargin': d_lmargin,
            'd_rmargin': d_rmargin,
            'd_top': d_top,
            'd_bottom': d_bottom,
            'top_size': top_size,
            'size_prop': size_prop,
            'width': self.width,
            'height': self.height,
            'nlines': self.nlines,
            'top_font': top_font,
            'font_prop': font_prop
        }
           
class LAPDFBlockLoader(BasePDFLoader):
    """Load `PDF` files using `PyMuPDF` into representative ."""

    def __init__(self, file_path: str) -> None:
        """Initialize with a file path."""
        try:
            import fitz  # noqa:F401
        except ImportError:
            raise ImportError(
                "`PyMuPDF` package not found, please install it with "
                "`pip install pymupdf`"
            )
        super().__init__(file_path)

    def load(self, **kwargs: Optional[Any]) -> List[Document]:
        """Load file."""

        parser = LAPDFBlockParser(text_kwargs=kwargs)
        blob = Blob.from_path(self.file_path)
        blocks = parser.parse(blob)
        return blocks

class LAPDFBlockParser(BaseBlobParser):
    """Parse `PDF` using `PyMuPDF`."""    

    def __init__(self, text_kwargs: Optional[Mapping[str, Any]] = None) -> None:
        """Initialize the parser.

        Args:
            text_kwargs: Keyword arguments to pass to ``fitz.Page.get_text()``.
        """
        self.text_kwargs = text_kwargs or {}

    def lazy_parse(self, blob: Blob) -> Iterator[Document]:
        """Lazily parse the blob."""
        import fitz
        #sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
        pos_err = 0.05
        
        with blob.as_bytes_io() as file_path:

            doc = fitz.open(file_path)  # open document

            feature_blocks = []
            dsizes = CumulativeTextFeature('font_sizes')
            dfonts = CumulativeTextFeature('fonts')
            all_spans = []
            for page in doc:
                d = page.get_textpage().extractDICT()
                for block in d.get('blocks'):
                    bfonts = CumulativeTextFeature('block_font')
                    bsizes = CumulativeTextFeature('block_font_size')
                    btext = ''
                    last_size = 0
                    for line in block.get('lines'):
                        for span in line.get('spans'):
                            all_spans.append(span)

                            text = span.get('text','')
                            text_size = span.get('size',0)
                            dsizes.add(text_size, text)
                            bsizes.add(text_size, text)

                            # if the text size changes more than 5%, add a space 
                            if abs(text_size - last_size) > pos_err:
                                btext += ' '
                            btext += text
                            last_size = text_size                             
                            
                            text_font = span.get('font',0)
                            dfonts.add(text_font, text)
                            bfonts.add(text_font, text)

                    pfb = LAPDF_FeatureBlock(
                        p=page.number,
                        x0=block.get('bbox')[0],
                        y0=block.get('bbox')[1],
                        x1=block.get('bbox')[2],
                        y1=block.get('bbox')[3],
                        text=btext,
                        nlines=len(block.get('lines')),
                        sizes=bsizes,
                        fonts=bfonts
                    )
                    feature_blocks.append(pfb)
                    
            # VERY SIMPLE APPROACH TO START WITH
            # Find the most common font size and use that to estimate the 
            # bounds of the page to exclude headers and footers
            #
            # Want to add a structured model to predict type of 
            # text blocks in the document  

            most_common_font = dfonts.read_most_common()
            most_common_size = dsizes.read_most_common()

            min_x0 = min([sp.get('bbox')[0]  for sp in all_spans if sp.get('size') == most_common_size])
            min_y0 = min([sp.get('bbox')[1]  for sp in all_spans if sp.get('size') == most_common_size])
            max_x1 = max([sp.get('bbox')[2]  for sp in all_spans if sp.get('size') == most_common_size])
            max_y1 = max([sp.get('bbox')[3]  for sp in all_spans if sp.get('size') == most_common_size])

            # Ideally, we would like to be able to classify the blocks into different types
            # For now, we will read the blocks in order unless their text seems to confirm to 
            # regexes to detect figure + table captions

            text = ''
            last_block = None   
            figure_regex = re.compile(r'^Fig(ure){0,1}\s*\d+\s*:', re.IGNORECASE)
            table_regex = re.compile(r'^Tab(le){0,1}\s*\d+\s*:', re.IGNORECASE)
            figs = []
            tables = []

            for pfb in feature_blocks:

                if pfb.x0 < min_x0 or pfb.y0 < min_y0 or pfb.x1 > max_x1 or pfb.y1 > max_y1:
                    continue    

                if figure_regex.match(pfb.t):
                    figs.append(pfb)
                    continue

                if table_regex.match(pfb.t):
                    tables.append(pfb)
                    continue
                
                if last_block is not None and \
                        (last_block.page != pfb.page or last_block.y1 < pfb.y0 - pos_err):
                    text += '\n\n'
                
                text += pfb.t
                last_block = pfb

            for pfb in figs:
                text += '\n\n' + pfb.t

            for pfb in tables:
                text += '\n\n' + pfb.t

        return Document(page_content=text, metadata=self.text_kwargs)

In [ ]:
#| export

class HuridocsPDFLoader(BasePDFLoader):
    """Load `PDF` files using `Huridocs` (https://github.com/huridocs/pdf_paragraphs_extraction)."""

    def __init__(self, file_path: str, host='localhost') -> None:
        """Initialize with a file path."""
        try:
            # check that the server's up and running
            url = 'http://'+host+':5051/info'
            r = requests.get(url)
        except Exception as e:
            raise Exception(
                "Huridocs `pdf_paragraphs_extraction` service not running. " + \
                "Please download it and run it from `https://github.com/GullyBurns/pdf_paragraphs_extraction`"
            )
        super().__init__(file_path)
        self.host = host

    def load(self, **kwargs: Optional[Any]) -> List[Document]:
        """Load file."""

        parser = HuridocsPDFParser(text_kwargs=kwargs, host=self.host)
        blob = Blob.from_path(self.file_path)
        blocks = parser.parse(blob)
        return blocks

class HuridocsPDFParser(BaseBlobParser):
    """Parse `PDF` using `Huridocs` (https://github.com/huridocs/pdf_paragraphs_extraction)."""    

    def __init__(self, text_kwargs: Optional[Mapping[str, Any]] = None, host='localhost') -> None:
        """Initialize the parser."""
        self.text_kwargs = text_kwargs or {}
        self.host = host

    def lazy_parse(self, blob: Blob) -> Iterator[Document]:
        """Lazily parse the blob."""
        import fitz
        #sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

        with blob.as_bytes_io() as file_path:

            # Use fitz to compute the document margins to exclude 
            # headers, footnotes, and page numbers
            try:
                doc = fitz.open(file_path)  # open document

                dsizes = CumulativeTextFeature('font_sizes')
                all_spans = []
                for page in doc:
                    d = page.get_textpage().extractDICT()
                    for block in d.get('blocks'):
                        for line in block.get('lines'):
                            for span in line.get('spans'):
                                text = span.get('text','')
                                if re.match('^[\d\s]+$', text): 
                                    continue
                                all_spans.append(span)
                                text_size = span.get('size',0)
                                dsizes.add(text_size, text)
                        
                most_common_size = dsizes.read_most_common()
                min_x0 = min([sp.get('bbox')[0]  for sp in all_spans if sp.get('size') == most_common_size])
                min_y0 = min([sp.get('bbox')[1]  for sp in all_spans if sp.get('size') == most_common_size])
                max_x1 = max([sp.get('bbox')[2]  for sp in all_spans if sp.get('size') == most_common_size])
                max_y1 = max([sp.get('bbox')[3]  for sp in all_spans if sp.get('size') == most_common_size])
            except Exception as e:
                # If fitz fails, use the Huridocs service to extract the text
                min_x0 = 0
                min_y0 = 0
                max_x1 = 10000
                max_y1 = 10000 

            url = 'http://'+self.host+':5051'
            files = {'file': file_path}
            r = requests.post(url, files=files)
            paragraphs = r.json().get('paragraphs', [])
            main_types = ['LIST', 'TITLE', 'TEXT', 'FORMULA']
            title = None
            docs = []
            text = ''
            for p in paragraphs:
                pg = p.get('page_number', 0)
                x0 = p.get('left',0)
                x1 = x0 + p.get('width',0)
                y0 = p.get('top',0)
                y1 = y0 + p.get('height',0)
                if y0<min_y0-5.0 or y1>max_y1+5.0:
                    continue
                if p.get('type') not in main_types:
                    continue
                if p.get('type') == 'TITLE':
                    if title is not None:
                        docs.append(Document(page_content=text, metadata=self.text_kwargs))
                        text = ''
                    title = p.get('text')
                text += p.get('text') + '\n'
            for p in paragraphs:
                if p.get('type') != 'FIGURE':
                    continue
                docs.append(Document(page_content=p.get('text')+'\n', metadata=self.text_kwargs))
            for p in paragraphs:
                if p.get('type') != 'TABLE':
                    continue
                docs.append(Document(page_content=p.get('text')+'\n', metadata=self.text_kwargs))
            return docs